In [35]:
import pandas as pd

In [36]:
base_vendas = pd.read_excel('bases/relatorio_notas.xlsx')
display(base_vendas)

,Numero nota fiscal,Data da venda,Código produto,Descrição do produto,Quantidade do produto,Valor unitário,Preço venda,Preço de custo,Valor da nota
0,566082,17/06/2025,33932,ENV 229X324 OU C/100 FILIPERSON,2.0,33.000000,27.476514,24.409041,3895.94
1,566082,17/06/2025,29367,GUARDANAPO SNOB FS 33X30 50F SANTHER,5.0,2.900000,2.499397,2.231242,3895.94
2,566082,17/06/2025,241,PASTA AZ OF LE TIGRADA FRAMA 356-11952,40.0,9.450000,8.863195,7.605561,3895.94
3,566082,17/06/2025,32608,CALCULADORA MESA 12 DIG MASTERPRINT 1086,6.0,20.870000,18.974913,17.051309,3895.94
4,566082,17/06/2025,18508,CAD BROCH UNIV STIFF AZ 96F JANDAIA,5.0,6.260000,5.680850,4.830296,3895.94
...,...,...,...,...,...,...,...,...,...
336734,403,03/01/2025,31327,CAIXA TERMICA PT 34L MOR,20.0,71.595537,71.595537,71.595537,NaN
336735,403,03/01/2025,31329,CAIXA TERMICA VM 18L MOR,52.0,44.910000,44.910000,44.910000,NaN
336736,403,03/01/2025,9552,CX ISOPOR 33L ISOPLAST,6.0,25.542042,25.998995,25.542042,NaN
336737,402,03/01/2025,6792,CX ISOPOR 3L ISOPLAST,24.0,2.780000,3.172271,2.780000,NaN


In [37]:
base_vendas.columns.tolist()

['Numero nota fiscal',
 'Data da venda',
 'Código produto',
 'Descrição do produto',
 'Quantidade do produto',
 'Valor unitário',
 'Preço venda',
 'Preço de custo',
 'Valor da nota']

In [38]:
base_vendas.isnull().sum()

Numero nota fiscal          0
Data da venda               0
Código produto              0
Descrição do produto        0
Quantidade do produto       0
Valor unitário              0
Preço venda                 0
Preço de custo           2224
Valor da nota            2745
dtype: int64

In [39]:
base_vendas.describe()

,Numero nota fiscal,Código produto,Quantidade do produto,Valor unitário,Preço venda,Preço de custo,Valor da nota
count,336739.000000,336739.000000,336739.000000,336739.000000,336739.000000,334515.000000,333994.000000
mean,480532.632603,22071.594784,15.671684,15.853359,13.092898,11.769463,2087.380438
std,180938.617261,11782.581172,85.143989,28.686929,23.675404,21.230737,3667.545385
min,401.000000,5.000000,-3125.000000,0.000000,0.000000,0.000000,0.000000
25%,544228.000000,11430.000000,2.000000,3.970000,3.187960,2.833333,502.070000
50%,551127.000000,24781.000000,5.000000,8.190000,6.671038,6.007989,1076.750000
75%,558739.000000,32304.000000,12.000000,18.820000,15.223790,13.530400,2341.640000
max,566082.000000,38229.000000,9764.000000,1169.000000,974.762703,880.340302,178016.720000


In [40]:
print("% dos Valores faltantes")
base_vendas.isnull().sum()/base_vendas['Código produto'].count() * 100

% dos Valores faltantes


Numero nota fiscal       0.000000
Data da venda            0.000000
Código produto           0.000000
Descrição do produto     0.000000
Quantidade do produto    0.000000
Valor unitário           0.000000
Preço venda              0.000000
Preço de custo           0.660452
Valor da nota            0.815171
dtype: float64

In [41]:
base_vendas.dropna(subset=['Valor da nota', 'Preço de custo'], axis=0, inplace=True)
base_vendas

,Numero nota fiscal,Data da venda,Código produto,Descrição do produto,Quantidade do produto,Valor unitário,Preço venda,Preço de custo,Valor da nota
0,566082,17/06/2025,33932,ENV 229X324 OU C/100 FILIPERSON,2.0,33.0000,27.476514,24.409041,3895.94
1,566082,17/06/2025,29367,GUARDANAPO SNOB FS 33X30 50F SANTHER,5.0,2.9000,2.499397,2.231242,3895.94
2,566082,17/06/2025,241,PASTA AZ OF LE TIGRADA FRAMA 356-11952,40.0,9.4500,8.863195,7.605561,3895.94
3,566082,17/06/2025,32608,CALCULADORA MESA 12 DIG MASTERPRINT 1086,6.0,20.8700,18.974913,17.051309,3895.94
4,566082,17/06/2025,18508,CAD BROCH UNIV STIFF AZ 96F JANDAIA,5.0,6.2600,5.680850,4.830296,3895.94
...,...,...,...,...,...,...,...,...,...
336430,496,01/04/2025,30643,TRAVESSEIRO SILK TOUCH 50X70 ALTENBURG,1.0,19.3375,15.539540,19.337500,0.00
336431,496,01/04/2025,35488,TNT TECIDO 50M 37G BR DECOFIX,60.0,46.4385,45.014518,46.438500,0.00
336432,496,01/04/2025,6620,BAND ISOPOR B3 BR C/400 COPOBRAS CRL003,-50.0,8.8327,8.832700,8.832700,0.00
336433,496,01/04/2025,30643,TRAVESSEIRO SILK TOUCH 50X70 ALTENBURG,-1.0,19.3375,15.539540,19.337500,0.00


In [42]:
print("% dos Valores faltantes")
base_vendas.isnull().sum()/base_vendas['Código produto'].count() * 100

% dos Valores faltantes


Numero nota fiscal       0.0
Data da venda            0.0
Código produto           0.0
Descrição do produto     0.0
Quantidade do produto    0.0
Valor unitário           0.0
Preço venda              0.0
Preço de custo           0.0
Valor da nota            0.0
dtype: float64

In [55]:
# Selecionar apenas as colunas numéricas
coluna_numericas = base_vendas.select_dtypes(include=['int64', 'float']).columns
# contar os valores zerados em cada coluna 
zeros = (base_vendas[coluna_numericas] == 0).sum()
# Exibir os resultados
print("Valores zerados por colunas")
print(zeros[zeros > 0]) # mostra apenas colunas com valores zero

Valores zerados por colunas
Valor unitário    268
Preço venda       142
Preço de custo      1
Valor da nota     606
dtype: int64
